<a href="https://colab.research.google.com/github/TADY25/Unil_Tissot/blob/main/Phrase_level_classification_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import csv

#Keras is used to create a prediciton model with neurons layers
import tensorflow as tf
from tensorflow import keras

!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6 MB 1.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [2]:
#Importing the data

data = pd.read_csv('training_data.csv')
data.drop(columns = ['id'], inplace = True)

In [3]:
def type_counter_unlabelled(df_fit, df_transform):
    nlp = spacy.load('fr_core_news_md')
    dif_typ = []
    
    for i in range(df_fit.shape[0]):
        doc = nlp(df_fit['sentence'].iloc[i])
        for j in doc:
            if j.pos_ not in dif_typ:
                dif_typ.append(j.pos_)
                
    res = []
    
    for i in range(df_transform.shape[0]):
        temp = np.zeros(len(dif_typ))
        doc = nlp(df_transform['sentence'].iloc[i])
        for j in doc:
            ind = dif_typ.index(j.pos_)
            temp[ind] += 1
        res.append(temp)
        
                
    return res

In [4]:
def type_counter(df):
    nlp = spacy.load('fr_core_news_md')
    dif_typ = []
    
    for i in range(df.shape[0]):
        doc = nlp(df['sentence'].iloc[i])
        for j in doc:
            if j.pos_ not in dif_typ:
                dif_typ.append(j.pos_)
                
    res = []
    
    for i in range(df.shape[0]):
        temp = np.zeros(len(dif_typ))
        doc = nlp(df['sentence'].iloc[i])
        for j in doc:
            ind = dif_typ.index(j.pos_)
            temp[ind] += 1
        res.append(temp)
        
                
    return res

In [5]:
def sk_vec(df):
    phrases = df['sentence'].tolist()
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(phrases).toarray()

In [6]:
def sk_vec_unlabelled(df_fit, df_transform):
  phrases_fit = df_fit['sentence'].tolist()
  phrases_transform = df_transform['sentence'].tolist()
  vectorizer = CountVectorizer()
  vectorizer.fit(phrases_fit)
  return vectorizer.transform(phrases_transform).toarray()

  

In [7]:
def vectorizer(df):
    tokenized_sentence = []
    nlp = spacy.load('fr_core_news_md')
    for sentence in df['sentence'].tolist():
        doc = nlp(sentence)
        tokenized_sentence.append(doc.vector)
        
    return tokenized_sentence            

In [8]:
def word_len_counter(df):
    res = []
    nlp = spacy.load('fr_core_news_md')
    for sentence in df['sentence'].tolist():
        temp = []
        doc = nlp(sentence)
        #doc=nb mots dans une phrase (ponct comprise)
        temp.append(len(doc))
        #sentence = nb de caractères dans une phrase(ponct comprise)
        temp.append(len(sentence))
        res.append(temp)
    return res
        

In [9]:
def prepare_output(df):
    res1 = df['difficulty'].map({'A1':0, 'A2':1, 'B1':2,'B2':3,'C1':4,'C2':5}).tolist()
    
    res2 = []
    mapping =  {'A1':0, 'A2':1, 'B1':2,'B2':3,'C1':4,'C2':5}
    for i in range(len(df)):
        temp = np.zeros(6)
        temp[mapping[df['difficulty'].iloc[i]]] = 1
        res2.append(temp)
        
    
    return res1, res2

In [10]:
def tf_accuracy(model, X,y):
    pred = model.predict(X)
    correct = 0
    for i in range(len(pred)):
        val =np.argmax(pred[i])
        if y[i][val] == 1:
            correct +=1
    return correct/len(y)

In [11]:
def evaluate(true, pred):
    precision = precision_score(true, pred, average="weighted")
    recall = recall_score(true, pred, average="weighted")
    f1 = f1_score(true, pred, average="weighted")
    print(confusion_matrix(true, pred))
    print(accuracy_score(true, pred))
    print(precision, recall, f1)

In [12]:
feature1 = type_counter(data)
feature2 = vectorizer(data)
feature3 = word_len_counter(data)
feature4 = sk_vec(data)

feature = np.concatenate((feature1, feature2),axis = 1)
feature = np.concatenate((feature, feature3), axis = 1)
feature = np.concatenate((feature, feature4), axis = 1)

In [13]:
feature = np.concatenate((feature1, feature2),axis = 1)
feature = np.concatenate((feature, feature3), axis = 1)
feature = np.concatenate((feature, feature4), axis = 1)

In [14]:
y1, y2 = prepare_output(data)

In [15]:
#X_train
#feature1_train = np.array(feature)
#y2_train = np.asarray(y2)
#y1_train = np.asarray(y1)
feature_train = np.array(feature[:4000])
feature4_train = np.array(feature4[:4000])
y2_train = np.asarray(y2[:4000])
y1_train = np.asarray(y1[:4000])
#X_test
feature_test = np.array(feature[4000:])
feature4_test = np.array(feature4[4000:])
y2_test = np.asarray(y2[4000:])
y1_test = np.asarray(y1[4000:])

In [16]:
print(len(feature4_test))

800


In [17]:
unlabelled = pd.read_csv('unlabelled_test_data.csv')
unlabelled.drop(columns = ['id'], inplace = True)

unlabelled_feature1=type_counter_unlabelled(data, unlabelled)
unlabelled_feature2=vectorizer(unlabelled)
unlabelled_feature3=word_len_counter(unlabelled)
unlabelled_feature4=sk_vec_unlabelled(data,unlabelled)

unlabelled_feature = np.concatenate((unlabelled_feature1, unlabelled_feature2),axis = 1)
unlabelled_feature = np.concatenate((unlabelled_feature, unlabelled_feature3), axis = 1)
unlabelled_feature = np.concatenate((unlabelled_feature, unlabelled_feature4), axis = 1)

unlabelled_feature = np.array(unlabelled_feature)

In [18]:
model = keras.Sequential([
    keras.layers.Dense(input_shape = feature[0].shape, units=512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=6, activation='softmax'),
    
    
])
#units1=32
print(model.summary())
model.compile(optimizer='adamax', 
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(np.asarray(feature), np.asarray(y2), epochs = 100, steps_per_epoch = 500, validation_split=0.01)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               7631360   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 7,634,438
Trainable params: 7,634,438
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
500/500 [==============================] - 23s 45ms/step - loss: 1.6148 - accuracy: 0.3518 - val_loss: 1.4114 - val_accuracy: 0.4167
Epoch 2/100
500/500 [==============================] - 22s 45ms/step - loss: 1.3235 - accuracy: 0.4421 - val_loss: 1.3590 - val_accuracy: 0.4792
Epoch 3/100
500

In [19]:
pred=model.predict(unlabelled_feature)
prediction_keras= []
for i in pred:
  prediction_keras.append(np.argmax(i))


reverse={0:"A1", 1:"A2", 2:"B1",3:"B2",4:"C1", 5:"C2"}

reversed_list=[reverse[k] for k in prediction_keras]
print(reversed_list)

with open("sample_submission.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     
     for item in reversed_list: 
      wr.writerow([item])

['A1', 'A2', 'A2', 'A1', 'C2', 'C1', 'A2', 'A2', 'C2', 'A2', 'A1', 'A2', 'B2', 'C1', 'A1', 'A2', 'C1', 'A1', 'A2', 'A1', 'C2', 'C1', 'A2', 'C1', 'B1', 'C2', 'A1', 'A1', 'C2', 'B1', 'A2', 'A2', 'A1', 'A1', 'A2', 'A2', 'C1', 'B2', 'A1', 'A2', 'A2', 'B2', 'C2', 'C1', 'C1', 'C2', 'B2', 'C1', 'A1', 'B1', 'C1', 'A1', 'B2', 'A2', 'A1', 'B2', 'C1', 'A2', 'C1', 'A2', 'B2', 'A2', 'C2', 'C1', 'C1', 'B2', 'B1', 'B1', 'B1', 'C2', 'A1', 'C1', 'B2', 'B1', 'C1', 'A2', 'B2', 'C1', 'B1', 'C1', 'A2', 'A2', 'B1', 'B1', 'B1', 'B2', 'B1', 'B2', 'C2', 'B1', 'C2', 'A2', 'C1', 'C2', 'A2', 'A2', 'A1', 'B1', 'A2', 'C2', 'B1', 'C1', 'A2', 'A2', 'C2', 'C2', 'B2', 'A1', 'A2', 'C1', 'B2', 'B1', 'C1', 'C2', 'C2', 'B1', 'C1', 'B1', 'A2', 'C2', 'A2', 'B1', 'C2', 'C1', 'A2', 'C2', 'C1', 'B2', 'C1', 'C1', 'A2', 'A2', 'C1', 'C2', 'B2', 'C1', 'A2', 'C2', 'B1', 'A2', 'A1', 'C1', 'C2', 'C2', 'C1', 'C1', 'A1', 'C1', 'B2', 'A2', 'B2', 'B2', 'A2', 'A2', 'A2', 'C2', 'C1', 'A1', 'A2', 'A1', 'A2', 'A1', 'A2', 'C1', 'C2', 'B2', 'A1

In [20]:
print(len(pred))

1200


In [21]:
classifier = LogisticRegression()
classifier=classifier.fit(feature4_train, y1_train)

y_pred_=classifier.predict(feature4_test)

evaluate(y1_test, y_pred)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


NameError: ignored

In [ ]:
classifier = KNeighborsClassifier()
classifier=classifier.fit(feature4_train, y1_train)


y_pred=classifier.predict(feature4_test)

evaluate(y1_test, y_pred)



In [ ]:
classifier = DecisionTreeClassifier()
classifier=classifier.fit(feature4_train, y1_train)


y_pred=classifier.predict(feature4_test)
  


evaluate(y1_test, y_pred)

In [ ]:


classifier = RandomForestClassifier()
classifier=classifier.fit(feature4_train, y1_train)


y_pred=classifier.predict(feature4_test)

evaluate(y1_test, y_pred)



In [22]:
model = keras.Sequential([
    keras.layers.Dense(input_shape = feature_train[0].shape, units=32, activation='relu'),
    keras.layers.Dropout(0.5),
    #keras.layers.Dense(units = 128, activation = "sigmoid"),
    #keras.layers.Dropout(0.6),
    keras.layers.Dense(units=6, activation='softmax')
    
    
])
#units1=32
print(model.summary())
model.compile(optimizer='adamax', 
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
keras_train=model.fit(np.asarray(feature4_train), np.asarray(y2_train), epochs = 50, steps_per_epoch = 500, validation_split=0.01)

accuracy_4=tf_accuracy(model, feature4_test, y2_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 32)                466752    
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                                 
Total params: 466,950
Trainable params: 466,950
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
500/500 [==============================] - 3s 5ms/step - loss: 1.7560 - accuracy: 0.2533 - val_loss: 1.7361 - val_accuracy: 0.2250
Epoch 2/50
500/500 [==============================] - 2s 5ms/step - loss: 1.6562 - accuracy: 0.3435 - val_loss: 1.6473 - val_accuracy: 0.3000
Epoch 3/50
500/500 [===

In [27]:
model = keras.Sequential([
    keras.layers.Dense(input_shape = feature_train[0].shape, units=32, activation='relu'),
    keras.layers.Dropout(0.5),
    #keras.layers.Dense(units = 128, activation = "sigmoid"),
    #keras.layers.Dropout(0.6),
    keras.layers.Dense(units=6, activation='softmax')
    
    
])
#units1=32
print(model.summary())
model.compile(optimizer='adamax', 
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(np.asarray(feature_train), np.asarray(y2_train), epochs = 50, steps_per_epoch = 500, validation_split=0.01)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                476960    
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_7 (Dense)             (None, 6)                 198       
                                                                 
Total params: 477,158
Trainable params: 477,158
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
500/500 [==============================] - 3s 5ms/step - loss: 1.7590 - accuracy: 0.2785 - val_loss: 1.5996 - val_accuracy: 0.3000
Epoch 2/50
500/500 [==============================] - 2s 4ms/step - loss: 1.5925 - accuracy: 0.3167 - val_loss: 1.4800 - val_accuracy: 0.3000
Epoch 3/50
500/500 [===

In [25]:

print(accuracy_4)

0.44


In [28]:

pred_1=model.predict(feature_test)
prediction_keras_1= []
for i in pred_1:
  prediction_keras_1.append(np.argmax(i))


reverse={0:"A1", 1:"A2", 2:"B1",3:"B2",4:"C1", 5:"C2"}

reversed_list=[reverse[k] for k in prediction_keras_1]



In [39]:
ids=[]

for i in range(len(reversed_list)):
  if prediction_keras_1[i]!=y1_test[i]:
    ids.append(4000+i)

comparaisons = pd.read_csv('training_data.csv')
comparaisons = comparaisons.iloc[4000:]
comparaisons = comparaisons.loc[comparaisons['id'].isin(ids)]    
comparaisons.head(20)

,id,sentence,difficulty
4002,4002,De nombreux reportages s'arrêtent sur ces père...,B2
4003,4003,On a beaucoup discuté de sa mère qui s'est cas...,A2
4005,4005,Je n'aime pas sortir la poubelle et je déteste...,A2
4006,4006,"Gutrie (Petty, 2000) citent les initiatives ma...",C2
4009,4009,Pas d'efforts violents et courts mais des effo...,B2
4015,4015,J'en ferai d'autres plus avancées.,B1
4016,4016,"Le gallinacé symbolise à la fois, la cérémonie...",B2
4021,4021,"Je déconseille le blanc, les harmonies de blan...",B2
4024,4024,"Ce n'était plus l'automne, c'était le printemps!",A2
4027,4027,Car Joe Manchin et Kyrsten Sinema s'étaient dé...,B2


In [35]:
print(y1_test)

[1, 4, 3, 1, 5, 1, 5, 2, 5, 3, 4, 4, 2, 5, 5, 2, 3, 4, 5, 5, 1, 3, 5, 4, 1, 0, 0, 3, 5, 4, 0, 2, 1, 2, 0, 0, 5, 4, 3, 0, 2, 1, 2, 2, 4, 5, 0, 2, 3, 4, 0, 1, 5, 5, 5, 0, 2, 3, 3, 4, 0, 2, 2, 4, 3, 3, 5, 3, 3, 2, 1, 1, 1, 4, 3, 0, 2, 3, 3, 4, 4, 3, 2, 2, 3, 0, 1, 3, 5, 4, 3, 5, 5, 1, 2, 1, 5, 5, 5, 2, 3, 4, 3, 3, 0, 2, 4, 3, 1, 2, 4, 2, 5, 4, 3, 5, 1, 4, 3, 3, 5, 5, 3, 4, 4, 2, 4, 1, 4, 2, 1, 3, 1, 5, 4, 0, 5, 4, 1, 0, 4, 2, 3, 2, 0, 5, 1, 1, 2, 3, 1, 2, 2, 0, 5, 0, 0, 1, 0, 3, 2, 3, 3, 5, 5, 2, 1, 5, 3, 1, 3, 4, 3, 0, 1, 0, 1, 1, 5, 2, 2, 0, 3, 0, 3, 0, 4, 3, 1, 3, 0, 0, 1, 2, 5, 1, 0, 5, 4, 1, 5, 3, 3, 1, 1, 5, 1, 5, 5, 1, 2, 0, 0, 4, 2, 4, 3, 2, 4, 1, 5, 0, 0, 3, 2, 1, 1, 0, 5, 1, 4, 3, 3, 1, 3, 0, 2, 2, 5, 4, 0, 2, 0, 3, 3, 0, 2, 5, 1, 0, 2, 2, 5, 4, 3, 4, 5, 5, 0, 4, 2, 4, 0, 4, 3, 3, 4, 0, 0, 1, 0, 3, 1, 2, 2, 3, 0, 3, 5, 3, 5, 2, 1, 2, 0, 5, 4, 3, 3, 5, 4, 4, 3, 5, 4, 1, 0, 4, 5, 3, 0, 1, 0, 3, 2, 0, 2, 1, 1, 4, 0, 1, 0, 4, 0, 4, 0, 3, 0, 5, 1, 0, 0, 1, 1, 2, 3, 0, 2, 3, 2, 3, 1, 